(normalization)=
# Normalization

```{tip}
To improve the
training stability, Llama{cite}`touvron2023llamaopenefficientfoundation` normalize the input of each
transformer sub-layer, instead of normalizing the
output. Using RMSNorm as the normalizing function.
```

```{figure} ../images/post-pre-norm.svg
```

## Batch Normalization

$$y = \frac{x - \mathbf{E}[x]}{\sqrt{\mathbf{Var}[x] + \epsilon}}*\gamma+\beta$$

$\mathbf{E}[x]$ and $\mathbf{Var}[x]$ are calculated over $(N, L)$ and they are vectors of size $C$, where $N$ denote batch size, $L$ denote sequence length and $C$ denote number of features.

In [1]:
import torch
from torch import nn

# (N, L, C)
batch, seq_len, num_features = 2, 3, 4
x = torch.randn(batch, seq_len, num_features).permute(0, 2, 1)  # BatchNorm1d requires (N, C, L) input
bn = nn.BatchNorm1d(num_features)

t1 = bn(x)
t2 = (x - x.mean(axis=[0, 2], keepdim=True)) / x.std(axis=[0, 2], unbiased=False, keepdim=True)
torch.allclose(t1, t2, atol=1e-3, rtol=0)

True

(layer_normalization)=
## Layer Normalization

$$y = \frac{x - \mathbf{E}[x]}{\sqrt{\mathbf{Var}[x] + \epsilon}}*\gamma+\beta$$

$\mathbf{E}[x]$ and $\mathbf{Var}[x]$ are calculated over $C$ and they are vectors of size $(N,L)$.

```{tip}
LayerNorm normalize each example and each position `independently`.
```

In [2]:
# (N, L, C)
batch, seq_len, num_features = 2, 3, 4
x = torch.randn(batch, seq_len, num_features)
ln = nn.LayerNorm(num_features)

t1 = ln(x)
t2 = (x - x.mean(axis=-1, keepdim=True)) / x.std(axis=-1, unbiased=False, keepdim=True)
torch.allclose(t1, t2, atol=1e-3, rtol=0)

True

## RMSNorm

$$y = \frac{x}{\sqrt{\mathbf{MeanSquare}[x] + \epsilon}}*\gamma$$

$\mathbf{MeanSquare}[x]$ is calculated over $C$ and is a vector of size $(N, L)$.

In [3]:
class RMSNorm(torch.nn.Module):
    """RMSNorm in Llama 3"""
    
    def __init__(self, dim: int, eps: float = 1e-6):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))

    def _norm(self, x):
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)

    def forward(self, x):
        # x shape: (N, L, C)
        # weight shape: (C,)
        output = self._norm(x.float()).type_as(x)
        return output * self.weight

## Why Layer Normalization

Layer Normalization (LN) is preferred over Batch Normalization (BN) in Transformer models for several key reasons.

1. **Batch Size Sensitivity**
    * BN normalizes across the batch dimension, meaning it computes statistics (mean and variance) using all samples in the mini-batch. This can lead to instability or poor performance when the batch size is small.
    * LN normalizes across the features within each individual example, independent of the batch.
2. **Sequence Length Variability**
    * BN assumes fixed-length inputs because it computes statistics over the entire batch, including all tokens in the sequence. However, in NLP tasks, sequences often have variable lengths, and padding is used to make them uniform. Padding tokens can distort the batch statistics.
    *  LN operates independently for each token in the sequence, normalizing across the feature dimensions for each token.
3. **Independence from Batch Statistics**
    *  Since BN relies on batch statistics (mean and variance), it introduces interdependence between examples in the same batch during training. This can lead to issues during inference when the model processes one example at a time (batch size of 1).
    *  LN does not depend on batch statistics, which ensures consistent behavior during both training and inference.
4. **Transformer Architecture Compatibility**
    * Transformers process sequences in parallel using self-attention mechanisms, where each token interacts with every other token in the sequence. The batch-wise nature of BN might interfere with these interactions.
5. **Training Stability**
    * BN can introduce instability during training due to its reliance on batch statistics, especially when the batch size is small or when there is significant variability in the input data. This can slow down convergence or lead to suboptimal solutions.